In [37]:
import nltk
import pandas as pd
import numpy as np
import sklearn
import csv
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

In [6]:
# moviereview_v2.csv was cleaned in hw 1

file = open('deception_data_converted_final_v2.csv', 'w')

with open('deception_data_converted_final.csv') as csvfile:
    
    reader = csv.reader(csvfile, delimiter = ',')
    file.write(','.join(next(reader)) +'\n') # header
    for row in reader:
        
        file.write(row[0] + ',' + row[1] + ',' + re.sub('\W+',' ', ''.join(row[2:])) +'\n') #data stripped and formatted

file.close()

In [7]:
csv_file = 'deception_data_converted_final_v2.csv'

df_review = pd.read_csv(csv_file, encoding = "ISO-8859-1")
df_review.dropna(inplace=True)
print(df_review.shape)

(92, 3)


In [8]:
df_review.head()

,lie,sentiment,review
0,f,n,Mike s Pizza High Point NY Service was very s...
1,f,n,i really like this buffet restaurant in Marsh...
2,f,n,After I went shopping with some of my friend ...
3,f,n,Olive Oil Garden was very disappointing I exp...
4,f,n,The Seven Heaven restaurant was never known f...


In [9]:
lieCol = df_review['lie']
sentCol = df_review['sentiment']
df_review.drop(['lie','sentiment'], axis=1, inplace=True)
df_review.head()

,review
0,Mike s Pizza High Point NY Service was very s...
1,i really like this buffet restaurant in Marsh...
2,After I went shopping with some of my friend ...
3,Olive Oil Garden was very disappointing I exp...
4,The Seven Heaven restaurant was never known f...


In [156]:
my_list = []
shortword = re.compile(r'\W*\b\w{1,3}\b')
my_list = [shortword.sub('', df_review.iloc[i,0]) for i in range(len(df_review))]


In [157]:
my_list[9]

' Yesterday went casino restaurant called NoFreeDrinks First thought just name later blackjack table when ordered attendant round Jack Coke instead saying Yeah sure attendant said preposterous have never paid drinks casino Clearly casino does understand that more people drink more they will loose Someone needs teach casino business '

In [172]:
count_vec = CountVectorizer(input="content", analyzer = 'word', stop_words = 'english', lowercase = True)
cv = count_vec.fit_transform(my_list)

# stolen from Dr. Gates
MyColumnNames=count_vec.get_feature_names()
VectorizedDF_Text=pd.DataFrame(cv.toarray(),columns=MyColumnNames)

In [173]:
print(VectorizedDF_Text.shape) # check size of vecorization before

threshold = 0 # threshold for drop
to_drop = [] # store indexes needed to be dropped

# id all indexes that don't meet threshold       
to_drop =  [i for i in range (len(VectorizedDF_Text.columns)) if sum(VectorizedDF_Text.iloc[:,i]) < threshold]  

# drop columns that don't meet threshold
VectorizedDF_Text.drop(VectorizedDF_Text.columns[to_drop],axis=1,inplace=True)

print(VectorizedDF_Text.shape)  # check size of vecorization after

(92, 1159)
(92, 1159)


In [174]:
VectorizedDF_Text.head()

,abruptly,absolutely,acceptable,accord,acknowledge,actual,actually,additional,adorable,affordable,...,wreck,write,written,wrong,wrote,yeah,yelp,yesterday,york,yuenan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
final_vector_df = VectorizedDF_Text.iloc[:,0:].apply(lambda x : x/len(VectorizedDF_Text)) # frequency of words

final_vector_df.insert(0, 'Lie', lieCol.to_frame())
final_vector_df.insert(1, 'Sentiment', sentCol.to_frame())

final_vector_df.head()

,Lie,Sentiment,abruptly,absolutely,acceptable,accord,acknowledge,actual,actually,additional,...,wreck,write,written,wrong,wrote,yeah,yelp,yesterday,york,yuenan
0,f,n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,f,n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,f,n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,f,n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,f,n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
train_df, test_df = train_test_split(final_vector_df, test_size=0.3)
print(train_df.shape)
print(test_df.shape)

(64, 1161)
(28, 1161)


In [191]:
train_df.insert(2, 'SL', train_df[['Sentiment', 'Lie']].apply(lambda x: ''.join(x), axis=1))
test_df.insert(2, 'SL', test_df[['Sentiment', 'Lie']].apply(lambda x: ''.join(x), axis=1))


nb_model_s = MultinomialNB()
nb_model_l = MultinomialNB()
nb_model_ls = MultinomialNB()
sentiment_l = train_df['Sentiment']
lie_l = train_df['Lie']
lieSent = train_df['SL']


train_NB = train_df.drop(['Lie','Sentiment','SL'],axis=1)

nb_model_s.fit(train_NB, sentiment_l)
nb_model_l.fit(train_NB, lie_l)
nb_model_ls.fit(train_NB, lieSent)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [192]:
actual_l = test_df['Lie']
actual_s = test_df['Sentiment']
actual_ls = test_df['SL']
test_NB = test_df.drop(['Lie','Sentiment','SL'],axis=1)

prediction_s = nb_model_s.predict(test_NB)
prediction_l = nb_model_l.predict(test_NB)
prediction_ls = nb_model_ls.predict(test_NB)

In [180]:
print(prediction_l)
print(actual_l)

['t' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't' 't'
 't' 't' 't' 't' 't' 't' 't' 't' 't' 't']
5     f
40    t
9     f
59    f
11    f
66    f
35    t
13    f
53    f
90    t
6     f
89    t
47    f
69    t
1     f
42    t
2     f
84    t
7     f
27    t
30    t
65    f
77    t
38    t
17    f
3     f
19    f
71    t
Name: Lie, dtype: object


In [181]:
print(prediction_s)
print(actual_s)

['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p'
 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
5     n
40    n
9     n
59    p
11    n
66    p
35    n
13    n
53    p
90    p
6     n
89    p
47    p
69    p
1     n
42    n
2     n
84    p
7     n
27    n
30    n
65    p
77    p
38    n
17    n
3     n
19    n
71    p
Name: Sentiment, dtype: object


In [193]:
print(prediction_ls)
print(actual_ls)

['pt' 'nf' 'nf' 'nf' 'nf' 'nf' 'pt' 'nf' 'pt' 'nf' 'nf' 'nf' 'nf' 'pt'
 'nf' 'pt' 'pt' 'pt' 'pt' 'pt' 'nf' 'pt' 'pt' 'pt' 'nf' 'pt' 'nf' 'nf']
67    pf
34    nt
31    nt
70    pt
16    nf
42    nt
52    pf
10    nf
23    nt
85    pt
59    pf
40    nt
41    nt
81    pt
82    pt
47    pf
51    pf
20    nf
50    pf
58    pf
6     nf
1     nf
89    pt
72    pt
3     nf
66    pf
45    nt
36    nt
Name: SL, dtype: object


In [194]:
cm_s = confusion_matrix(actual_l, prediction_l)
cm_l = confusion_matrix(actual_s, prediction_s)
cm_sl = confusion_matrix(actual_ls, prediction_ls)

In [196]:
print(cm_s)
print()
print(cm_l)
print()
print(cm_sl)

[[ 9  5]
 [10  4]]

[[11  3]
 [ 4 10]]

[[4 0 0 2]
 [7 0 0 1]
 [1 0 0 7]
 [3 0 0 3]]


In [184]:
np.round(nb_model_l.predict_proba(test_NB))

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])